In [2]:
from envs.environment import Environment
from envs.test_env import Electric_Car
from models.baseline import BaselineModel
from models.DQN import DQNModel
import pandas as pd
import numpy as np
from tqdm import tqdm 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [12]:
%load_ext autoreload
%autoreload 3
from envs.test_env import Electric_Car
from models.DQN import DQNModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
test_env = Electric_Car('data/train.xlsx')

In [14]:
dqn = DQNModel(test_env, replay_capasity=500)

In [15]:
test_env.observation()

array([2.500e+01, 2.431e+01, 1.000e+00, 0.000e+00, 1.000e+00, 1.000e+00,
       2.007e+03, 1.000e+00])

In [26]:
df = pd.DataFrame(columns=['battery_level', 'price', 'hour', 'day_of_week', 'day_of_year', 'month', 'year', 'available', 'action', 'reward'])
for row in range(27000):
    action = np.random.random(1)
    state, reward, term, trunc, act = test_env.step(action)
    df.loc[len(df.index)] = [*state, act, reward]
    if term or trunc:
        print('env stop')

env stop


IndexError: index 1096 is out of bounds for axis 0 with size 1096

In [23]:
df = df.sample(frac=1).reset_index(drop=True)

In [27]:
df.shape

(26303, 10)

In [21]:
df.reward.max()

-0.0

In [25]:
import sklearn
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import RandomForestRegressor

In [23]:
X = df[df.columns[:-1]]
y = df.reward

In [26]:
forest = RandomForestRegressor().fit(X, y)

In [44]:
val_env = Electric_Car('data/validate.xlsx')

In [31]:
def pick_action_forest(env):
    max_action = -2
    max_reward = float('-inf')
    obs = env.observation()
    for action in np.linspace(-1, 1, 21):
        pred = forest.predict([[*obs, action]])
        if pred > max_reward:
            max_reward = pred
            max_action = action
    return max_action, max_reward

In [45]:
done = False
rewards = []
for i in tqdm(range(17519)):
    # action, reward = pick_action_forest(val_env)
    action = (np.random.random()-0.5)*2
    obs, _, tr, te, _ = val_env.step(action)
    rewards.append(reward)
    done = tr or te

print(sum(rewards))

100%|██████████| 17519/17519 [00:00<00:00, 17562.02it/s]

[0.]


In [9]:
env = Electric_Car('data/train.xlsx')

In [10]:
model = DQNModel(env, replay_capasity=10, batch_size=4)

In [8]:
model.train(1)

training on cpu


1it [00:00, 39.58it/s]00:00<?, ?it/s]
  0%|          | 0/1 [00:00<?, ?it/s]

None


ValueError: Action must be between -1 and 1

In [5]:
from models.DeepQLearning import DDQNAgent
test_env = Electric_Car('data/validate.xlsx')
model = DDQNAgent('data/validate.xlsx', device='cpu', epsilon_decay=10_000, epsilon_start=1.0,
                   epsilon_end=.05, discount_rate=.99, lr=5e-4, buffer_size=50_000)
model.init_envs(test_env)
model.load('ddqn_tn.pth')

Please wait, the experience replay buffer will be filled with random transitions
Initialization with random transitions is done!


In [4]:
model.load('ddqn_tn.pth')